In [1]:
import pandas as pd
import re
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
df_movies = pd.read_csv("regressionPreprocessing.csv")
print("Length after import: " + str(len(df_movies)))
df_movies = df_movies.fillna(0)
df_movies.head(5)

Length after import: 7517


,budget,id,runtime,vote_count,History,Western,Music,Family,Comedy,Drama,...,orig_ml,orig_es,orig_bn,orig_ab,orig_wo,orig_ca,orig_ru,orig_id,orig_is,director
0,30000000,862,81.0,5415,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1709
1,65000000,8844,104.0,2413,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1645
2,60000000,949,170.0,1886,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2338
3,58000000,710,130.0,1194,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2231
4,98000000,1408,119.0,137,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2812


In [3]:
features_to_remove = ['+18','id','actors','spokenLanguages','productionCountries','productionCompanies','original_language','hasHomepage','part_of_collection']
for i in features_to_remove:
    if i in df_movies.columns:
        df_movies = df_movies.drop(columns=i)
print(df_movies.head(5))
df_movies.columns

     budget  runtime  vote_count  History  Western  Music  Family  Comedy  \
0  30000000     81.0        5415      0.0      0.0    0.0     1.0     1.0   
1  65000000    104.0        2413      0.0      0.0    0.0     1.0     0.0   
2  60000000    170.0        1886      0.0      0.0    0.0     0.0     0.0   
3  58000000    130.0        1194      0.0      0.0    0.0     0.0     0.0   
4  98000000    119.0         137      0.0      0.0    0.0     0.0     0.0   

   Drama  Foreign  ...  orig_ml  orig_es  orig_bn  orig_ab  orig_wo  orig_ca  \
0    0.0      0.0  ...      0.0      0.0      0.0      0.0      0.0      0.0   
1    0.0      0.0  ...      0.0      0.0      0.0      0.0      0.0      0.0   
2    1.0      0.0  ...      0.0      0.0      0.0      0.0      0.0      0.0   
3    0.0      0.0  ...      0.0      0.0      0.0      0.0      0.0      0.0   
4    0.0      0.0  ...      0.0      0.0      0.0      0.0      0.0      0.0   

   orig_ru  orig_id  orig_is  director  
0      0.0     

Index(['budget', 'runtime', 'vote_count', 'History', 'Western', 'Music',
       'Family', 'Comedy', 'Drama', 'Foreign', 'Action', 'Horror', 'Mystery',
       'Romance', 'Animation', 'Documentary', 'Thriller', 'TV Movie', 'Crime',
       'Science Fiction', 'Fantasy', 'War', 'Adventure', 'rating', '18+',
       'orig_lv', 'orig_el', 'orig_fa', 'orig_tl', 'orig_ta', 'orig_th',
       'orig_mn', 'orig_zh', 'orig_te', 'orig_kk', 'orig_zu', 'orig_et',
       'orig_mr', 'orig_eu', 'orig_sv', 'orig_no', 'orig_pl', 'orig_cs',
       'orig_cy', 'orig_bs', 'orig_de', 'orig_lo', 'orig_xx', 'orig_ko',
       'orig_hu', 'orig_sr', 'orig_da', 'orig_pt', 'orig_nl', 'orig_en',
       'orig_it', 'orig_tr', 'orig_hr', 'orig_cn', 'orig_ka', 'orig_ar',
       'orig_ja', 'orig_0', 'orig_hi', 'orig_ro', 'orig_af', 'orig_sk',
       'orig_fr', 'orig_fi', 'orig_he', 'orig_uk', 'orig_bg', 'orig_ml',
       'orig_es', 'orig_bn', 'orig_ab', 'orig_wo', 'orig_ca', 'orig_ru',
       'orig_id', 'orig_is', 'director']

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

#features = features.dropna()

# separate features and target variable
rating = df_movies['rating'] # weight
features = df_movies.drop(columns=['rating'])

# create a train/test split
features_train, features_test, rating_train, rating_test = train_test_split(
    features, rating, test_size=0.4, random_state=42)

print("Train: " + str(len(features_train)) + " Features and " + str(len(rating_train)) + " Ratings")
print("Test: " + str(len(features_test)) + " Features and " + str(len(rating_test)) + " Ratings")

Train: 4510 Features and 4510 Ratings
Test: 3007 Features and 3007 Ratings


In [25]:
import matplotlib.pyplot as plt
# machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

features = features
rating = rating

# Splitting the dataset into the Training set and Test set
# I have a fairly large dataset of +- 4000 entries, so I'm going with 10% test data
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(features, rating, test_size = 0.1)

# create and fit a linear regression
linear_reg = LinearRegression()
linear_reg.fit(features_train, target_train)
target_pred_linreg = linear_reg.predict(features_test)

print("Scores:")
mse_1 = mean_squared_error(target_test, target_pred_linreg)
r2_1 = r2_score(target_test, target_pred_linreg)
print("MSE:", mse_1)
print("RMSE:", sqrt(mse_1))
print("R^2:", r2_1)
print()


# #This regressor was picked with gridsearch over many parameters - took 4 hours
# from xgboost import XGBRegressor
# regressor = XGBRegressor(colsample_bytree= 0.6, gamma= 0.7, max_depth= 4, min_child_weight= 5,
#                          subsample = 0.8, objective='reg:squarederror')
# regressor.fit(X, y)
# y_pred = regressor.predict(X_test)

# from sklearn.metrics import r2_score
# score = r2_score(y_test, y_pred)
# print("Score: ", score)

# fig, ax = plt.subplots()
# ax.scatter(y_test, y_pred)
# ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
# ax.set_xlabel('Measured rating')
# ax.set_ylabel('Predicted rating')
# plt.title('Measured versus predicted rating')
# plt.ylim((1, 5))   # set the ylim to bottom, top
# plt.xlim(1, 5)     # set the ylim to bottom, top
# plt.show()

Scores:
MSE: 0.37081681311630804
RMSE: 0.6089472991288393
R^2: -0.019130634385431167

